## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Balancer

https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum'

6-Month Average Daily Volume

In [3]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [4]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [5]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [6]:
df['dailyVolumeUSD'].astype(float).mean()

108344346.07359475

6-Month Average TVL

In [18]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/balancer.csv")

In [19]:
lma

,Unnamed: 0,Date,Timestamp,Balancer,Balancer.1,Balancer.2,Balancer.3,Balancer.4,Balancer.5,Balancer.6,Balancer.7,Balancer.8,Balancer.9,Balancer.10,Balancer.11
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,ethereum,polygon,arbitrum,Total,ethereum,polygon,arbitrum,Total,ethereum,polygon,arbitrum,Total
2,NaN,NaN,NaN,TVL,TVL,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USDT,USDT,USDT,USDT,USDT,USDT,USDT,USDT
4,NaN,24/03/2020,1.585008e+09,7833.948843186513,NaN,NaN,7833.948843186513,7833.948843186513,NaN,NaN,7833.948843186513,7808.045244,NaN,NaN,7808.045244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,NaN,14/07/2022,1.657757e+09,1216269288.255335,39158513.836108,11592946.567114,1267020748.6585572,1216269288.255335,39158513.836108,11592946.567114,1267020748.6585572,1216269288.255335,39158513.836108,11592946.567114,1267020748.6585572
846,NaN,15/07/2022,1.657843e+09,1224966514.939901,38574127.700004,12525253.81965,1276065896.4595551,1224966514.939901,38574127.700004,12525253.81965,1276065896.4595551,1224966514.939901,38574127.700004,12525253.81965,1276065896.4595551
847,NaN,16/07/2022,1.657930e+09,1246713654.1502051,37749738.749862134,12571579.731327616,1297034972.6313949,1246713654.1502051,37749738.749862134,12571579.731327616,1297034972.6313949,1245468185.964241,37712026.723139,12559020.710617,1295739233.3979971
848,NaN,17/07/2022,1.658016e+09,1293302760.413176,39876224.489226,13134777.303393,1346313762.205795,1293302760.413176,39876224.489226,13134777.303393,1346313762.205795,1293302760.413176,39876224.489226,13134777.303393,1346313762.205795


In [20]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

2368622794.775219

Largest Pool Concentration

In [7]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_11865/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [8]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x61d5dc44849c9c87b0856a2a311536205c96c7fd,Balancer v2 Comfy,20398.48614786238573090796947158718,12712670.72880471859715758724473423,1619037254,12285749
1,0x647c1fd457b95b75d0972ff08fe01d7d7bda05df,Balancer v2 Balancer 80 BAL 20 WETH,16.23566920498595271783626895059565,1693.037280185061522342577267617469,1619044051,12286257
2,0x0297e37f1873d2dab4487aa67cd56b58e2f27875,Balancer v2 Balancer 50 WBTC 50 WETH,5156.024834313822093418773152127843,9234.50843576363334143132482008345,1619044189,12286276
3,0xc6a5032dc4bf638e15b4a66bc718ba7ba474ff73,Balancer v2 Balancer 60 WETH 40 DAI,1811.215195230905578103102836670492,7544.756476919556580341633367559084,1619188473,12297008
4,0x3a19030ed746bd1c3f2b0f996ff9479af04c5f0a,Balancer v2 Balancer 50 WBTC 49 USDC 1 cUNI,0.0007169165524228923529056882647308386,68.87368912339535592605351237377125,1619219562,12299382
...,...,...,...,...,...,...
694,0x06b4007e0d903c8187e1f9ee6daef6695c42b0be,Balancer v2 80BANK-20WETH,1438.956881065522392462975508484237,0,1658007989,15156223
695,0xa6feeee62b839f1389d53eee2ca426bf6a8cf689,Balancer v2 SKI Copper LBP,113759.225764443641933729038627995,731138.1461260704958349792201987475,1658084231,15161866
696,0xba52a93a53f8161636f8233f36d32a0ce4ae844b,Balancer v2 30WBTC-50PAXG-20WETH,1818.658982473901785561903972173198,67.40893777652525257097968155344637,1658109628,15163799
697,0x13f2f70a951fb99d48ede6e25b0bdf06914db33f,Balancer v2 Balancer USDD-FRAX-USDC Stablepool,2.001065,0,1658133482,15165609


In [9]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.35027259731828947

In [10]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.8136186971545017

Liquidity Pools with > $1mn TVL

In [11]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

37

Monthly Active Users

In [12]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [13]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [14]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [15]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [16]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [17]:
np.mean(monthactiveusers) #can chart this easy

19874.714285714286

Implied slippage